In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import nltk

from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

In [ ]:
testset = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
trainset = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

trainset.head()

In [ ]:
trainset.info()

In [ ]:
trainset.shape

In [ ]:
trainset.isnull().sum()

In [ ]:
less_lable =[]
for i in range(len(trainset)):
    lbl = 0
    less_lable.append(lbl)
    
    
toxic_lable =[]
for i in range(len(trainset)):
    lbl = 1
    toxic_lable.append(lbl)


In [ ]:
less = trainset['less_toxic']
toxic = trainset['more_toxic']

lessdata = pd.DataFrame(list(zip(less,less_lable)), columns=['text','label'])

toxicdata = pd.DataFrame(list(zip(toxic,toxic_lable)), columns=['text','label'])

traindata = lessdata.append(toxicdata, ignore_index=True)


traindata.head()

In [ ]:
def preprocess(text):
    
    #tokenization
    text = str(text)
    tokenised_text = text_to_word_sequence(text,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
    
    #stop word removal  
    stop_words = set(stopwords.words('english'))
    stopwordremove = [i for i in tokenised_text if not i in stop_words]
    
    #number removal
    text = ' '.join(stopwordremove)
    numberremoval = ''.join(x for x in text if not x.isdigit())
    
    #stemming
    stemmer = PorterStemmer()
    
    stemm = text_to_word_sequence(numberremoval)
    stemtext = ' '.join(stemmer.stem(word) for word in stemm)
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()

    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    lem_input = nltk.word_tokenize(stemtext)
    lem_text= ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lem_input])
    
    
    return lem_text
    
    


In [ ]:
traindata['text']=traindata['text'].apply(preprocess)  

In [ ]:
text = traindata['text']
label = traindata['label']
text.head(20)

In [ ]:
testset.head()

In [ ]:
testset.shape

In [ ]:
testset['text']=testset['text'].apply(preprocess)
test = testset['text']


In [ ]:
rfc = Pipeline([('tfidf', TfidfVectorizer()),
                ('rfc', RandomForestClassifier(n_estimators=100)),
               ])

rfc.fit(text, label)

test_predict = rfc.predict(test)

train_accuracy = round(rfc.score(text, label)*100)

print("Random Forest Classifier Train Accuracy Score : {}% ".format(train_accuracy ))



In [ ]:
y_prdict = rfc.predict_proba(test)[:, 1]

In [ ]:
commentid = testset['comment_id']

In [ ]:
submission = pd.DataFrame(list(zip(commentid, y_prdict)),
               columns =['comment_id', 'score'])
submission.head(20)


In [ ]:
submission.to_csv('submission.csv', index=False)